# Linear Solver Performance: XDG Stokes, Single Core
### Part 1, Benchmark Setup and Execution

In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 7/22/2022 7:12:22 AM


In [2]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
wmg.Init("WIP-k-LinslvPerfSer");
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

Project name is set to 'WIP-k-LinslvPerfSer'.
Opening existing database '\\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer'.


In [4]:
// for examination on the local workstation, 
//OpenDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes");

In [5]:
/*
// extract the control object to reproduce specific simulation on local workstation
string dest = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\BenchControls";
foreach(var s in wmg.Sessions) {
    string name = s.Name;
    Console.Write(name + ": ");
    try {
        var ctrl = s.GetControl();
        var txt = ctrl.Serialize();
        
        string DestPath = System.IO.Path.Combine(dest, name + ".obj");
        System.IO.File.WriteAllText(DestPath, txt);
        
        Console.WriteLine("written");
    } catch(Exception e) {
        Console.WriteLine(e.Message);
    }
}
*/

In [6]:
//var ss = wmg.Sessions.Where(sess => sess.Name.Contains("XdgStokes"));
//foreach(var s in ss)
//    s.Delete(true);

## Utility definitions

In [7]:
static class Utils {
    // DOF per cell in 3D for one variable
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

Note: The dimension of the domain $(-1,1)^3$ are assumed to be **centimeters**!
In this benchmark, realistic physical values are used, i.e. densities and viscosities of 
- water for Phase A (inside the droplet)
- air for Phase B (surounding)
The droplet is assumed to be in the millimeter range; therefore, all lenght-, area-, and volume-related properties 
must also be set in centimeters, e.g. the density of water is $10^{-3} \text{kg}/\text{cm}^3$.

In [8]:
int[] Resolutions_3D = new int[] { 8, 16, 24, 32, 48, 64 };
//int[] Resolutions_3D = new int[] { 2 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] zNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "-XdgStokes_J" + J);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(GridName)); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            if (Math.Abs(X[0] - (-1)) <= 1.0e-8)
                return "wall_left";
            if (Math.Abs(X[0] - (+1)) <= 1.0e-8)
                return "wall_right";
            if (Math.Abs(X[1] - (-1)) <= 1.0e-8)
                return "wall_front";
            if (Math.Abs(X[1] - (+1)) <= 1.0e-8)
                return "wall_back";
            if (Math.Abs(X[2] - (-1)) <= 1.0e-8)
                return "wall_top";
            if (Math.Abs(X[2] - (+1)) <= 1.0e-8)
                return "wall_bottom";
            throw new ArgumentException("unknown wall");
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Opening existing database 'C:\BoSSStests\WIP-k-LinslvPerfSer'.
Found Grid: { Guid = 332447a7-b0a7-4cde-a75a-3b615a81aaac; Name = WIP-k-LinslvPerfSer-XdgStokes_J512; Cell Count = 512; Dim = 3 }
Found Grid: { Guid = a7d28a70-9959-4417-80ae-13290b53ab44; Name = WIP-k-LinslvPerfSer-XdgStokes_J4096; Cell Count = 4096; Dim = 3 }
Found Grid: { Guid = 4c63f3fc-0b08-4711-8f54-e535594e9183; Name = WIP-k-LinslvPerfSer-XdgStokes_J13824; Cell Count = 13824; Dim = 3 }
Found Grid: { Guid = e21a1da6-8dd3-423e-8fc2-2054eb72ba69; Name = WIP-k-LinslvPerfSer-XdgStokes_J32768; Cell Count = 32768; Dim = 3 }
Found Grid: { Guid = ddbfd64f-6bb3-4184-8f9b-1f674fa10231; Name = WIP-k-LinslvPerfSer-XdgStokes_J110592; Cell Count = 110592; Dim = 3 }
Found Grid: { Guid = 1efa192f-ecf1-45ca-9f1f-d17d8db22954; Name = WIP-k-LinslvPerfSer-XdgStokes_J262144; Cell Count = 262144; Dim = 3 }


In [9]:
grids

#0: { Guid = 332447a7-b0a7-4cde-a75a-3b615a81aaac; Name = WIP-k-LinslvPerfSer-XdgStokes_J512; Cell Count = 512; Dim = 3 }
#1: { Guid = a7d28a70-9959-4417-80ae-13290b53ab44; Name = WIP-k-LinslvPerfSer-XdgStokes_J4096; Cell Count = 4096; Dim = 3 }
#2: { Guid = 4c63f3fc-0b08-4711-8f54-e535594e9183; Name = WIP-k-LinslvPerfSer-XdgStokes_J13824; Cell Count = 13824; Dim = 3 }
#3: { Guid = e21a1da6-8dd3-423e-8fc2-2054eb72ba69; Name = WIP-k-LinslvPerfSer-XdgStokes_J32768; Cell Count = 32768; Dim = 3 }
#4: { Guid = ddbfd64f-6bb3-4184-8f9b-1f674fa10231; Name = WIP-k-LinslvPerfSer-XdgStokes_J110592; Cell Count = 110592; Dim = 3 }
#5: { Guid = 1efa192f-ecf1-45ca-9f1f-d17d8db22954; Name = WIP-k-LinslvPerfSer-XdgStokes_J262144; Cell Count = 262144; Dim = 3 }


In [10]:
//var ss = wmg.Sessions.Where(sess => sess.Name.Contains("XdgStokes") && sess.Name.Contains("exp_Kcycle_schwarz"));
//foreach(var s in ss)
//    s.Delete(true);

## Setup Control Object for a Solver Run

In [11]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [12]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Setup of Parameter Study

Polynomial degrees to test:

In [13]:
int[] PolyDegS = new int[] {2, 3 ,5};

Solvers which we want to instrument:

In [14]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.direct_pardiso,
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.pMultigrid
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [15]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.direct_pardiso:
        case LinearSolverCode.direct_mumps:
        return 1100000; // 1.1 Million for direct solvers at maximum
    
        default: 
        return 3000000; // Up to 3 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [16]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [17]:
List<XNSE_Control> controls = new List<XNSE_Control>();
controls.Clear();
foreach(LinearSolverCode solver in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {

    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*3 + Np_p;
    if(J*DOF >  GetMaxAllowedDOF(solver))
        continue;
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add(C);
       
    string caseName = string.Format("XdgStokes-J{0}_p{1}_{2}", J, k, solver);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    // Species A: Water; Species B: Air
    C.PhysicalParameters.rho_A             = 1.0; //1e-3; //     kg / cm³
    C.PhysicalParameters.rho_B             = 1.0; //1.2e-6; //   kg / cm³
    C.PhysicalParameters.mu_A              = 1.0; //1e-5; //      kg / cm / sec
    C.PhysicalParameters.mu_B              = 1.0; //17.1e-8; //   kg / cm / sec
    C.PhysicalParameters.Sigma             = 72.75e-3; // kg / sec²   
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
    // Dont know
    // ============
    
    double r     = 0.5;
    double nonsp = 0.5;

    C.AddInitialValue("Phi", new Formula($"X => (X[0]/{r*nonsp}).Pow2() + (X[1]/{r}).Pow2() + (X[2]/{r}).Pow2() - 1", false));
    
    C.LSContiProjectionMethod = BoSSS.Solution.LevelSetTools.ContinuityProjectionOption.None;
    //C.CutCellQuadratureType   = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.ComputeEnergyProperties = false;

    
    // Solver Stuff
    // ============
    
    C.LinearSolver           = solver.GetConfig();
    if(C.LinearSolver is IterativeSolverConfig isc) {
        isc.ConvergenceCriterion = 1e-8;
    }
    C.NoOfMultigridLevels = 100;
    C.TracingNamespaces = "BoSSS.Solution";
    C.LevelSet_ConvergenceCriterion     = 1e-6;

    //C.Option_LevelSetEvolution                          = LevelSetEvolution.FastMarching;
    C.AdvancedDiscretizationOptions.SST_isotropicMode   = SurfaceStressTensor_IsotropicMode.Curvature_Projected;
    //C.AdvancedDiscretizationOptions.ViscosityMode       = ViscosityMode.Standard;
    C.AdvancedDiscretizationOptions.FilterConfiguration = CurvatureAlgorithms.FilterConfiguration.NoFilter;

    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
    //C.dtFixed = 0.01;
}
}
}

setting up: XdgStokes-J512_p2_direct_pardiso
setting up: XdgStokes-J4096_p2_direct_pardiso
setting up: XdgStokes-J13824_p2_direct_pardiso
setting up: XdgStokes-J512_p3_direct_pardiso
setting up: XdgStokes-J4096_p3_direct_pardiso
setting up: XdgStokes-J13824_p3_direct_pardiso
setting up: XdgStokes-J512_p5_direct_pardiso
setting up: XdgStokes-J4096_p5_direct_pardiso
setting up: XdgStokes-J512_p2_exp_gmres_levelpmg
setting up: XdgStokes-J4096_p2_exp_gmres_levelpmg
setting up: XdgStokes-J13824_p2_exp_gmres_levelpmg
setting up: XdgStokes-J32768_p2_exp_gmres_levelpmg
setting up: XdgStokes-J512_p3_exp_gmres_levelpmg
setting up: XdgStokes-J4096_p3_exp_gmres_levelpmg
setting up: XdgStokes-J13824_p3_exp_gmres_levelpmg
setting up: XdgStokes-J32768_p3_exp_gmres_levelpmg
setting up: XdgStokes-J512_p5_exp_gmres_levelpmg
setting up: XdgStokes-J4096_p5_exp_gmres_levelpmg
setting up: XdgStokes-J13824_p5_exp_gmres_levelpmg
setting up: XdgStokes-J512_p2_exp_Kcycle_schwarz
setting up: XdgStokes-J4096_p2_e

Total number of simulations:

In [18]:
controls.Count

41

In [19]:
// Assert that the location of the fluid phases is as desired:
NUnit.Framework.Assert.Negative(controls[0].InitialValues["Phi"].Evaluate(new double[]{0,0.0,0}, 0.0), "phase A (negative) must be inside");
NUnit.Framework.Assert.Positive(controls[0].InitialValues["Phi"].Evaluate(new double[]{0,4.0,0}, 0.0), "phase B (positive) must be inside");

In [20]:
// Basic checks on the multigrid configuration
foreach(var ctrl in controls) {
    NUnit.Framework.Assert.Greater(ctrl.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
    if(ctrl.LinearSolver is OrthoMGSchwarzConfig osc)
        NUnit.Framework.Assert.Greater(osc.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
}

In [21]:
//string path = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\Xstokes";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, ctrl.SessionName + ".obj"));
//}

## Launch Jobs

Use the default queue defined on this machine:

In [22]:
ExecutionQueues

index,RuntimeLocation,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
0,win\amd64,C:\BoSSStests,False,MSHPC-Gitrunner-HighPrio,dotnet,FDY\jenkinsci,DC2,[ fdygitrunner ],Highest,True,"[ C:\BoSSStests, C:\Users\jenkinsci\ ]"
1,win\amd64,\\fdygitrunner\BoSSStests,False,MSHPC-Gitrunner-DefaultTest,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, fdygitrunner, hpcluster3, hpccluster2 ]",Normal,True,[ \\fdygitrunner\BoSSStests ]
2,win\amd64,\\fdygitrunner\ValidationTests,True,MSHPC-AllNodes,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, hpccluster2, hpcluster3, fdygitrunner ]",Normal,True,[ \\fdygitrunner\ValidationTests ]
3,win\amd64,\\fdygitrunner\ValidationTests,True,MSHPC-FastNodes,dotnet,FDY\jenkinsci,DC2,[ hpcluster3 ],Normal,True,[ \\fdygitrunner\ValidationTests ]


In [23]:
var myBatch = GetDefaultQueue();
myBatch

RuntimeLocation,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
win\amd64,\\fdygitrunner\ValidationTests,True,MSHPC-AllNodes,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, hpccluster2, hpcluster3, fdygitrunner ]",Normal,True,[ \\fdygitrunner\ValidationTests ]


In [24]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var j = ctrl.CreateJob();
    j.RetryCount = 4;
    j.Activate(myBatch);
    //ctrl.RunBatch();
}

 Submitting: XdgStokes-J512_p2_direct_pardiso
Note: found 2 successful deployment(s), but job is configured to require a successful result session ('this.SessionReqForSuccess' is true), and none is found. 0 sessions correlated to this job fount in total.
Deploying job XdgStokes-J512_p2_direct_pardiso ... 
Deploying executables and additional files ...
Deployment directory: \\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul22_071310
copied 50 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.

 Submitting: XdgStokes-J4096_p2_direct_pardiso
Note: found 2 successful deployment(s), but job is configured to require a successful result session ('this.SessionReqForSuccess' is true), and none is found. 0 sessions correlated to this job fount in total.
Deploying job XdgStokes-J4096_p2_direct_pardiso ... 
Deploying executables and additional files ...
Deployment directory: \\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Ju

Deploying executables and additional files ...
Deployment directory: \\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul22_072012
copied 50 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.

 Submitting: XdgStokes-J32768_p3_exp_gmres_levelpmg
Loading session 26140c17-10be-4564-9ac6-28c9d5e0bf98 failed with message 'Could not find file '\\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer\sessions\26140c17-10be-4564-9ac6-28c9d5e0bf98\Session.info'.'
Note: found 2 successful deployment(s), but job is configured to require a successful result session ('this.SessionReqForSuccess' is true), and none is found. 0 sessions correlated to this job fount in total.
Deploying job XdgStokes-J32768_p3_exp_gmres_levelpmg ... 
Deploying executables and additional files ...
Deployment directory: \\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul22_072040
copied 50 files.
   written file: control.obj
   copied 'win\amd64' runtime.
d

Deploying job XdgStokes-J4096_p5_exp_Kcycle_schwarz ... 
Deploying executables and additional files ...
Deployment directory: \\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul22_072700
copied 50 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.

 Submitting: XdgStokes-J13824_p5_exp_Kcycle_schwarz
 ------------ MSHPC FailedOrCanceled; original Failed
Note: Job was deployed (3) number of times, all failed; RetryCount is 4, so try once more.
Hint: want to re-activate the job.
Job is FailedOrCanceled, but retry count is set to 4 and only 3 tries yet - trying once more...
Deploying job XdgStokes-J13824_p5_exp_Kcycle_schwarz ... 
Deploying executables and additional files ...
Deployment directory: \\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul22_072724
copied 50 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.

 Submitting: XdgStokes-J512_p2_pMultigrid
Note: found 2 success

### Wait for Completion and Check Job Status

In [25]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

Error at loading of session:7a98a708-5f7f-401f-b5b8-a3476c5ed297
Error at loading of session:ef8e0c51-4532-477d-b0d8-7576867b84b4
Error at loading of session:ddc81fe3-0ad4-424f-936f-7d52d748406e
Error at loading of session:b67642b7-2e7c-40d0-b0dc-c0be8d47d830
Error at loading of session:d362b867-4e05-4651-93f5-f477533bafea
Error at loading of session:1b9837cb-c70d-4be8-ad72-4ee0811db8f6
 ------------ MSHPC FailedOrCanceled; original Failed
 ------------ MSHPC FailedOrCanceled; original Failed
 ------------ MSHPC FailedOrCanceled; original Failed
 ------------ MSHPC FailedOrCanceled; original Failed
 ------------ MSHPC FailedOrCanceled; original Failed
 ------------ MSHPC FailedOrCanceled; original Failed
All jobs finished.


In [26]:
wmg.AllJobs

#0: XdgStokes-J512_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	XdgStokes-J512_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: XdgStokes-J4096_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	XdgStokes-J4096_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: XdgStokes-J13824_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	XdgStokes-J13824_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: XdgStokes-J512_p3_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	XdgStokes-J512_p3_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationT

In [27]:
wmg.Sessions.Where(sess => sess.Name.StartsWith("XdgStokes_J"))

In [28]:
//wmg.Sessions[3].Export().WithSupersampling(2).Do()

In [29]:
var NoSuccess = controls.Select(ctrl => ctrl.GetJob()).Where(job => job.Status != JobStatus.FinishedSuccessful).ToArray();
NoSuccess

#0: XdgStokes-J4096_p5_direct_pardiso: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: XdgStokes-J32768_p3_exp_gmres_levelpmg: Unknown (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: XdgStokes-J512_p5_exp_gmres_levelpmg: Unknown (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: XdgStokes-J4096_p5_exp_gmres_levelpmg: Unknown (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#4: XdgStokes-J13824_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#5: XdgStokes-J4096_p3_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#6: XdgStokes-J32768_p3_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#7: XdgStokes-J4096_p5_exp_Kcycle_schwarz: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\Valida

In [30]:
// In the case of some failed job, print the directory name for further inspection:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + ((fail.LatestDeployment?.DeploymentDirectory?.FullName) ?? " no deployment directory"));
    //Console.WriteLine(fail.LatestDeployment);

XdgStokes-J4096_p5_direct_pardiso: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul22_071649
XdgStokes-J32768_p3_exp_gmres_levelpmg: Unknown (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul22_072040
XdgStokes-J512_p5_exp_gmres_levelpmg: Unknown (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul22_072108
XdgStokes-J4096_p5_exp_gmres_levelpmg: Unknown (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul22_072137
XdgStokes-J13824_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\WIP-k-LinslvPerfSer-XNSE_Solver2022Jul

In [31]:
/*
string PathOffset = @"C:\Users\jenkinsci\Desktop\LinSlvPerfFail-20apr22";
foreach(var fail in NoSuccess) {
    var C = fail.GetControl();
    C.savetodb = false;
    C.SaveToFile(System.IO.Path.Combine(PathOffset, fail.Name + ".obj"));
    
    string Stdout = fail.Stdout;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stdout.txt"), Stdout);
    
    string Stderr = fail.Stderr;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stderr.txt"), Stderr);
}
*/

In [32]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("XdgStokes") && Si.SuccessfulTermination == false);
FailedSessions

#0: WIP-k-LinslvPerfSer	XdgStokes-J13824_p5_pMultigrid*	07/22/2022 07:34:11	94285ee1...
#1: WIP-k-LinslvPerfSer	XdgStokes-J13824_p5_exp_Kcycle_schwarz*	07/22/2022 07:27:38	662bfec8...
#2: WIP-k-LinslvPerfSer	XdgStokes-J32768_p3_exp_Kcycle_schwarz*	07/22/2022 07:26:11	bc8d90f9...
#3: WIP-k-LinslvPerfSer	XdgStokes-J4096_p5_exp_Kcycle_schwarz*	07/22/2022 07:27:14	32fe5f41...
#4: WIP-k-LinslvPerfSer	XdgStokes-J4096_p3_exp_Kcycle_schwarz*	07/22/2022 07:25:21	01bb6696...
#5: WIP-k-LinslvPerfSer	XdgStokes-J13824_p5_exp_gmres_levelpmg*	07/22/2022 07:22:15	72cd6a08...
#6: WIP-k-LinslvPerfSer	XdgStokes-J32768_p3_exp_gmres_levelpmg*	07/22/2022 07:20:54	c01775c9...
#7: WIP-k-LinslvPerfSer	XdgStokes-J4096_p5_exp_gmres_levelpmg*	07/22/2022 07:21:52	a6ea3b65...
#8: WIP-k-LinslvPerfSer	XdgStokes-J512_p5_exp_gmres_levelpmg*	07/22/2022 07:21:22	54fbb1b5...
#9: WIP-k-LinslvPerfSer	XdgStokes-J4096_p5_direct_pardiso*	07/22/2022 07:17:03	52a24090...


#### Asserting Success:

Remark: since this is currently (22 Apr. 2022) work-in-progress, we allow for some jobs to fail; 
At this intermediate milestone, I want to record (by the means of tests) what **is already working**.
Thereby, I hope I won't break the working cases while trying to fix the failing ones.

In [33]:
var prelim_allowedFails = new[] { 
"XdgStokes-J13824_p5_exp_Kcycle_schwarz", 
"XdgStokes-J4096_p5_exp_Kcycle_schwarz",
"XdgStokes-J32768_p3_exp_Kcycle_schwarz", 
"XdgStokes-J13824_p3_exp_Kcycle_schwarz",
"XdgStokes-J32768_p2_exp_Kcycle_schwarz",
//
"XdgStokes-J13824_p5_exp_gmres_levelpmg",
"XdgStokes-J4096_p5_exp_gmres_levelpmg",
"XdgStokes-J32768_p3_exp_gmres_levelpmg",
//
"XdgStokes-J4096_p5_direct_pardiso",
//
"XdgStokes-J32768_p3_pMultigrid",	
"XdgStokes-J13824_p5_pMultigrid",	
"XdgStokes-J4096_p5_pMultigrid",
"XdgStokes-J32768_p2_pMultigrid",	
"XdgStokes-J13824_p3_pMultigrid",	
"XdgStokes-J4096_p3_pMultigrid",
"XdgStokes-J13824_p2_pMultigrid",	
"XdgStokes-J4096_p2_pMultigrid"	
 };

In [34]:
//foreach(var si in FailedSessions)
//   si.Delete(true);

In [35]:
NUnit.Framework.Assert.Zero(NoSuccess.Where(job => !prelim_allowedFails.Contains(job.Name)).Count(), "Some Jobs Failed");

Unhandled exception: NUnit.Framework.AssertionException:   Some Jobs Failed
  Expected: 0
  But was:  2

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.Zero(Int32 actual, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Conditions.cs:line 484
   at Submission#37.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchE

In [36]:
NUnit.Framework.Assert.Zero(FailedSessions.Where(s => !prelim_allowedFails.Contains(s.Name)).Count(), "Some Sessions did not terminate successfully.");

Unhandled exception: NUnit.Framework.AssertionException:   Some Sessions did not terminate successfully.
  Expected: 0
  But was:  2

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.Zero(Int32 actual, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Conditions.cs:line 484
   at Submission#38.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exce